一 模型构造

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.2778, -0.1021, -0.0520, -0.0211, -0.1326, -0.0159,  0.1438, -0.1401,
          0.0721,  0.0444],
        [-0.2566, -0.2121, -0.0091, -0.0175, -0.1238,  0.0227,  0.1633, -0.1198,
          0.0325, -0.0963]], grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-0.0226, -0.0591, -0.0891,  0.2256,  0.0452, -0.2165, -0.0765,  0.0559,
          0.2951, -0.1310],
        [-0.0585, -0.0286, -0.1175,  0.2980,  0.1516, -0.2134,  0.0278,  0.0265,
          0.2546, -0.2128]], grad_fn=<AddmmBackward0>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[block] = block
    
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1516,  0.0408, -0.1080,  0.0733,  0.1557,  0.0094, -0.0577,  0.0442,
          0.1637, -0.1183],
        [ 0.1904,  0.1679, -0.0609,  0.0543,  0.0373, -0.0069,  0.0549,  0.0173,
          0.1321, -0.2740]], grad_fn=<AddmmBackward0>)

在正向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(0.0945, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2258, grad_fn=<SumBackward0>)

二、参数管理

In [10]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0455],
        [0.1236]], grad_fn=<AddmmBackward0>)

参数访问

In [11]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1199,  0.0268, -0.0125, -0.2899,  0.1091,  0.0084,  0.1930,  0.2125]])), ('bias', tensor([-0.0631]))])


目标参数

In [12]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0631], requires_grad=True)
tensor([-0.0631])


In [13]:
net[2].weight.grad is None

True

一次性访问所有参数

In [14]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [15]:
net.state_dict()['2.bias'].data

tensor([-0.0631])

从嵌套块收集参数

In [16]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3041],
        [0.3041]], grad_fn=<AddmmBackward0>)

In [17]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [18]:
rgnet[0][1][0].bias.data

tensor([ 0.4021,  0.3663, -0.3856, -0.4042,  0.0053, -0.2639,  0.2988,  0.4735])

内置初始化

In [19]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0138, -0.0058,  0.0033,  0.0194]), tensor(0.))

In [20]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]


(tensor([1., 1., 1., 1.]), tensor(0.))

对某些块应用不同的初始化方法

In [21]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)


tensor([ 0.2328, -0.0242,  0.1942,  0.2108])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [22]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  0.0000, -5.1746,  0.0000],
        [-0.0000, -0.0000,  0.0000, -0.0000]], grad_fn=<SliceBackward0>)

参数绑定

In [23]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),shared, nn.ReLU(),shared, nn.ReLU(),nn.Linear(8, 1))

net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


三、自定义层

构造一个没有任何参数的自定义层

In [24]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

In [25]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [26]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [27]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-9.3132e-10, grad_fn=<MeanBackward0>)

带参数的层

In [28]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)


In [29]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-1.0628,  1.3427, -0.4321],
        [ 0.1731, -0.6089, -0.5324],
        [ 1.0739,  0.6376, -0.4829],
        [ 0.5208,  0.3886, -0.6827],
        [ 0.4105,  0.2402,  0.9722]], requires_grad=True)

In [30]:
linear(torch.rand(2, 5))

tensor([[1.5324, 0.0000, 0.7767],
        [2.7487, 0.0000, 0.4573]])

In [31]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[4.2470],
        [0.0000]])

四、读写文件

加载和保存张量

In [33]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [34]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

存储一个张量列表，然后把他们读回内存

In [35]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

写入或读取从字符串映射到张量的字典

In [36]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

加载和保存模型参数

In [37]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)


将模型的参数存储为一个叫做'mlp.params‘的文件

In [38]:
torch.save(net.state_dict(), 'mlp.params')

In [39]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [40]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])